<a href="https://colab.research.google.com/github/mhmd2015/ImageProcessing1UIC/blob/main/DIP_UIC_MNIST_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install idx2numpy

  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=ee60fda678b581b9648a2e81c35c7d05563eaa36f7dafcf39d42bea9d35d5104
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy


In [9]:
import numpy as np

from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist

In [7]:
import idx2numpy

def load_data(images_path, labels_path):
    images = idx2numpy.convert_from_file(images_path)
    labels = idx2numpy.convert_from_file(labels_path)
    return images, labels

In [ ]:
# train_images, train_labels = load_data('train-images-idx3-ubyte', 'train-labels-idx1-ubyte')

# test_images, test_labels = load_data('t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte')

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, -1)).astype('float32') / 255  # Flatten and normalize

In [10]:





# (b) Split data and thresholding
# Using all 60000 for training and 10000 for testing as specified in the problem
train_images = np.where(train_images > 128, 1, 0)
test_images = np.where(test_images > 128, 1, 0)

# Flatten the images for the GMM
train_images = train_images.reshape((60000, 28*28))
test_images = test_images.reshape((10000, 28*28))

# (c) Model each class using GMM
gmm_classifiers = {}
for label in range(10):
    data = train_images[train_labels == label]
    gmm = GaussianMixture(n_components=2, covariance_type='diag').fit(data)
    gmm_classifiers[label] = gmm

# (d & e) Compute accuracy
def classify(image):
    scores = []
    for label, gmm in gmm_classifiers.items():
        score = np.log((train_labels == label).mean()) + gmm.score_samples([image])
        scores.append(score)
    return np.argmax(scores)

predicted_train_labels = [classify(img) for img in train_images]
predicted_test_labels = [classify(img) for img in test_images]

train_accuracy = accuracy_score(train_labels, predicted_train_labels)
test_accuracy = accuracy_score(test_labels, predicted_test_labels)

print(f"Training accuracy: {train_accuracy:.2f}")
print(f"Testing accuracy: {test_accuracy:.2f}")

11490434/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  .fit(X)
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  .fit(X)
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  .fit(X)
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  .fit(X)
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to dup

Training accuracy: 0.11
Testing accuracy: 0.11


In [11]:
# Initialize variables for mean, covariance, and priors for each class
means = np.zeros((10, 28*28))
covariances = np.zeros((10, 28*28))
priors = np.zeros(10)

# Estimate mean, covariance, and prior for each class
for digit in range(10):
    images_of_digit = train_images[train_labels == digit]
    means[digit, :] = np.mean(images_of_digit, axis=0)
    covariances[digit, :] = np.var(images_of_digit, axis=0) + 1e-6  # Adding small constant for stability
    priors[digit] = len(images_of_digit) / len(train_images)

def bayesian_classifier(x):
    """Classify an image using Bayes' rule."""
    posteriors = np.zeros(10)
    for digit in range(10):
        likelihood = np.exp(-0.5 * np.sum(((x - means[digit]) ** 2) / covariances[digit]))
        posteriors[digit] = likelihood * priors[digit]
    return np.argmax(posteriors)

# Classify test images
predicted_labels = np.array([bayesian_classifier(x) for x in test_images])

# Calculate accuracy
accuracy = np.mean(predicted_labels == test_labels)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

Accuracy on the test set: 9.80%
